In [1]:
from sht.sht import DirectSHT
import sht.interpolation as interp
import numpy as np

%load_ext autoreload
%autoreload 2

In [2]:
# We can very quickly calculate Ylm values. As an example, compute them up to lmax=Nl
Nl  = 500
Nx  = 1024
theta_samples = (np.arange(Nx)+0.5)/Nx
sht = DirectSHT(Nl,Nx)

In [3]:
# Now let's explore cubic spline interpolation of one of these alm's
ell = 100
m = 33
Ylm = sht.Yv[sht.indx(ell, m), :]

In [4]:
# Let's generate some mock data
Nrandoms = int(1e7)
theta_data = np.random.uniform(0,np.pi,Nrandoms)
w_i = np.random.uniform(0,1, Nrandoms)

# We will want to sort the data in ascending order of theta
sorted_indices = np.argsort(theta_data)
theta_data_sorted = theta_data[sorted_indices]
w_i_sorted = w_i[sorted_indices] # The observation weights for each galaxy/random

In [5]:
# Calculate theta_data-theta_sample[i] for each theta_data
which_spline_idx = np.digitize(theta_data_sorted, theta_samples) - 1
digitized_diffs = (theta_data_sorted - theta_samples[which_spline_idx])

In [6]:
# We now sum up all the w_p (theta_data_p-theta_sample[i])^n in each spline region i, for n in {0,1,2,3}, and a galaxy/random labeled by p
summed_0_diffs = interp.get_sum(theta_samples, theta_data_sorted,
                         w_i_sorted * np.ones_like(digitized_diffs))
summed_1_diffs = interp.get_sum(theta_samples, theta_data_sorted,
                         w_i_sorted * digitized_diffs)
summed_2_diffs = interp.get_sum(theta_samples, theta_data_sorted,
                         w_i_sorted * digitized_diffs**2)
summed_3_diffs = interp.get_sum(theta_samples, theta_data_sorted,
                         w_i_sorted * digitized_diffs**3)

print('Note that these take up a negligible amount of memory: {} Mb'.format(4*summed_0_diffs.nbytes/1e6))

Note that these take up a negligible amount of memory: 0.032768 Mb


In [7]:
# Now the name of the game is to calculate the 4 spline coefficients in each region, multiply them by the summed_..._diffs
# and finally, sum over all splines
a_lm = np.sum(interp.cubic_spline(theta_samples, Ylm, np.arange(1024),
                           summed_0_diffs, summed_1_diffs, summed_2_diffs, summed_3_diffs))

# After that, we just loop over ell and m. And this is actually trivially parallelizable

In [8]:
%%timeit
np.sum(interp.cubic_spline(theta_samples, Ylm, np.arange(1024),
                    summed_0_diffs, summed_1_diffs, summed_2_diffs, summed_3_diffs))

18.4 ms ± 43.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
